# THIS WILL MAKE ME PRIMERZ
Calling from info in Tm_Calc to make a function that returns tm/primer sequence! 

In [1]:
# Importing my tools 
import pandas as pd
import numpy as np

from itertools import product

from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq 

import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
df = pd.read_csv('updated_constitutive_promoters_from_izaiah.csv')

In [ ]:
df["Promoter B4 5\' UTR"]

In [3]:
df[["TAIR_ID","Promoter 2kb upstream from ATG "]].to_csv("test_input.csv", index=False)
df[["TAIR_ID","Promoter B4 5\' UTR"]].to_csv("test_input_utr.csv", index=False)

In [34]:
test_df = df[["TAIR_ID","Promoter 2kb upstream from ATG "]]

# Defining fxns 
My thought process: Get DNA --> make all primer options fwd (15-25 bp)/Tms and put into dataframe --> Same for rev --> Comparison/brute force out the min difference of tms --> return sequences that gave Tms 

In [12]:
# Tm fxn    
def mytm(seq):
    return mt.Tm_NN(seq, dnac1=500, dnac2=0, selfcomp=False, Na=0, K=50, Tris=25, Mg=2, dNTPs=0.2, saltcorr=5)

In [13]:
# Fxn Tms for fwd sequence   
def tmfwd(dna):
    fwddna_list = []
    fwdtms_list = []
    count = 14
    while count < 61:
        count += 1
        if mytm(dna[0:count]) > 55 and mytm(dna[0:count]) < 72:
                fwddna_list.append(dna[0:count])
                fwdtms_list.append(mytm(dna[0:count]))
    seqcolm = pd.Series(fwddna_list,name='Sequence')
    tmcolm = pd.Series(fwdtms_list,name='Tms').apply(np.round).astype(int)
    return pd.concat([seqcolm,tmcolm],axis=1)

In [14]:
# Fxn Tms for rev 
def tmrev(dna): 
    seqdnas = Seq(dna)
    revcompl = str(seqdnas.reverse_complement())
    revdna_list = []
    revtms_list = []
    count = 14
    while count < 61:
        count += 1
        if mytm(dna[0:count]) > 55 and mytm(dna[0:count]) < 72:
                revdna_list.append(dna[0:count])
                revtms_list.append(mytm(dna[0:count]))
    revseqcolm = pd.Series(revdna_list,name='Sequence')
    revtmcolm = pd.Series(revtms_list,name='Tms').apply(np.round).astype(int)
    return pd.concat([revseqcolm,revtmcolm],axis=1)

In [15]:
# Main fxn
def primerfxn(dna):
    df3 = tmrev(dna) 
    Tmsrev = df3['Tms'].to_list()
    df2 = tmfwd(dna)
    Tmsfwd = df2['Tms'].to_list()
    # more definitions
    allposs = list(product(Tmsfwd,Tmsrev))
    finddiff = []
    for pair in allposs:
        finddiff.append(abs(pair[0]-pair[1]))
    findmindiff = allposs[np.argmin(finddiff)]
    fwdprim_list = (df2[df2['Tms'] == findmindiff[0]]['Sequence'].to_list()[0])
    fwdprimstms_list = (df2[df2['Tms'] == findmindiff[0]]['Tms'].to_list()[0])
    revprim_list = (df3[df3['Tms'] == findmindiff[1]]['Sequence'].to_list()[0])
    revprimstms_list = (df3[df3['Tms'] == findmindiff[1]]['Tms'].to_list()[0])
    return fwdprim_list, revprim_list ,revprimstms_list, fwdprimstms_list

In [6]:
# The test site 
mydna = 'caggtcttctctgtagctctgttacttctatcacagttatcgggtatttgagaaaaaagagttagctaaaatgaatttctccatataatcatggtttactacaggtttacttgattcgcgttagctttatctgcatccaaagttttttccatgatgttatgtcatatgtgataccgttactatgtttataactttatacagtctggttcactggagtttctgtgattatgttgagtacatactcattcatcctttggtaactctcaagtttaggttgtttgaattgcctctgttgtgatacttattgtctattgcatcaatcttctaatgcaccaccctagactatttgaacaaagagctgtttcattcttaaacctctgtgtctccttgctaaatggtcatgctttaatgtcttcacctgtctttctcttctatagatatgtagtcttgctagatagttagttctacagctctcttttgtagtcttgttagagagttagttgagatattacctcttaaaagtatccttgaacgctttccggttatgaccaatttgttgtagctccttgtaagtagaacttactgggaccagcgagacagtttatgtgaatgttcatgcttaagtgtcgaacgtatctatctctactatagctctgtagtcttgttagacagttagttttatatctccatttttttgtagtcttgctagttgagatattacctcttctcttcaaagtatccttgaacgctcaccggttatgaaatctctacactatagctctgtagtcttgctagatagttagttctttagctctctttttgtagcctagttctttagctctccttttgtagccttgctacagagtaagatgggatattacctccttgaacgctctccggttatgaccaatttgttgtagctccttgtaagtagaacttaggatagagtgagtcaactttaagaaagaacctagtatgtggcataaccagattgcaggctctgtctcggctacagtaacgtaactctatagctctttgttttgttcagaaagaaccagtgattggatgattcgtccttagaaactggacctaacaacagtcattggctttgaaatcaagccacaacaatgcctatatgaaccgtccatttcatttatccgtttcaaaccagcccattacatttcgtcccattgataaccaaaagcggttcaatcagattatgttttaattttaccaaattctttatgaagtttaaattatactcacattaaaaggattattggataatgtaaaaattctgaacaattactgattttggaaaattaacaaatattctttgaaatagaagaaaaagcctttttccttttgacaacaacatataaaatcatactcccattaaaaagattttaatgtaaaattctgaatataagatattttttacaacaacaaccaaaaatatttatttttttccttttttacagcaacaagaaggaaaaacttttttttttgtcaagaaaaggggagattatgtaaacagataaaacagggaaaataactaaccgaactctcttaattaacatcttcaaataaggaaaattatgatccgcatatttaggaagatcaatgcattaaaacaacttgcacgtggaaagagagactatacgctccacacaagttgcactaatggtacctctcacaaaccaatcaaaatactgaataatgccaacgtgtacaaattagggttttacctcacaaccatcgaacattctcgaaacattttaaacagcctggcgccatagatctaaactctcatcgaccaatttttgaccgtccgatggaaactctagcctcaacccaaaactctatataaagaaatcttttccttcgttattgcttaccaaatacaaaccctagccgccttattcgtcttcttcgttctctagttttttcctcagtctctgttcttagatcccttgtagtttccaaatcttccgataaaa'
primerfxn(mydna)
mydna1 = 'TGCATTTTTTGTAGCTTCTTGGTCTGTAGCCAAGTATTTCCCAGACAACGAGCCGCCTCCAAGAGGGGAATAAGCAAGCAAGCCAACATTGCAATTTTTTGGGTGGCATACTTCTACCAGATCAActgtggttttaacccaaaacaaaagagtttagaacatgctcctccgacacacttgctctcaaggtattctagaaagatttacCTTCATACCGACATCTAACTAGCAAGCTGTAACCATTCTGGATGCTCACAATCTTTGGTAGTCCTTCGAGTTTTGCTGTGTTAACAAACTCCGTCACTCCGTATGAAGTTTCATTCGAGACACCGATATAGCGAACctgttgcagaattcgaggaacacttcaaaactctgtgtttgaggttatataactcaatataacatctgatttttaaggagtagcactacCTTTCCTTCAACTATGAGATCCTGAAAGGCTCTTAGTTGCTCAGCGAATGGCACACTAGGTCTCCATTTCGACGTTTCATAGTAAAAATCACCGAAGAGTGGTACGTATCGATCTGGCctggactgaaaaaaagatcagtgtcatgattttctgacatttgatgatgaaaaacgaaatattttgcgcatgaaatagaaagacacttgctcaaacaatgtagaatatgagttaagacataaagcaaaaataaggcgatgttacCAGTGTATTTGAAGCAAGTCAATGTAATCAGTTCCAAGGCGCTTAAGACTTTTCTCAACACTTTCTTTGATATTAGCAGCATCTACACGCAGAATCTCACCACTGTCCCTTATGTAAGCTGACCTTTCTGAGTACCCACATACTTTAGTTGCCAAAACTATctgtatacagaaacaatgaacatcaatatcaatttttataacttcgattaggttttgctgatatcatatacattcgtgaccaatatgcatccaaagtatgataaatggttgatacCTTGTCACGTTGCTGAGACTTCAACCAGCTACTGATATAAAGATCCGTTTTACCTTGCGTCTCCTTCTTCATCGGTATGGGATActggaaataaacaatgatacttgaaaatttaccacagactaagcagattccagaaagctaaacacaacagtgctaaattcaccagaattatatataaagtataaaacattcacataaacacttgcaatcaaattcaccaggcaagaagagaaagatgaacaatgaagataaaaactcacAGCTTCAGCAGTGTCAATGCAATTGATGCCCTCTTCAATTGCATAACTCAGCATCTCATGAGATTCTTTCTCAGTATTTTGCTCCCCAAATGTCATctgaaattccaaatttcatcaaaacttcaaaattgaacaatggactacaattgcaggagataaagagagattgttcttacAGTGCCCATAGTAACTTCGCTGATGTTGAGATCGGAATCTCCTAATTTCCTATACTCCATAGCATTCTTCGTCGACGCTTTCGCCGAAACTCTCAATATTCTCCTCCGAGCATTCGGCGATAGAGTAACACGGCGGCGCGTGAATGCCAAACGAAAGACTCCTCGAATGTTACTGCTGCTGACCACCGGAGTGACCGTTGAGTAGGTGACGTAAGATGACGCCATaccggaggttgtcgtggacgtggttcttctcgccttatcgccaccattatctactgttttgattcacgtatttgaatactctgggctaaacgatttgggcctaatgtaagaggcccattagagtcaaaaatatatattgaataatatcttgttttcctaatatgataggtctaatctcagatttcgatttatttcttcgagataatatgattatattagatgacgtggcggtacatccttcgatatcttcgataaatacgcaagagtgagcttcacggtcagacaaaagctgactctacgctctctcaaaagtcgaatatcccaaattcgatccgttttccggcgataatctccgatctagatccg'
primerfxn(mydna1)
mydna2 = 'attcaaagatgtgtaatgggaggaacacagaaaatttcctcataacattagcaagaaaagagaatgaaatctcatttttttcgaggatgtcatgccaatgtccataagtactaacagaagcaaagaccataatatactacaaaattttccacaaatttatctggatcaaacaggcatggtgattaggatgcagcaaacgattcgtacCTGTAAAGTTGCCAGAAACAAATGCACGAGAAGCGTCTctgagcgaaagaatgacatggaaaaacacttcccatcagctacaaacatgaaagattaacaagaagaagatagaagaacatatgagaatctctccacataatttttaacaaaacaaacCTTCCAGCAAAATGGTTGTACCCTCCTCCACTACCGTAATGAAATTTTCCTTTCGTCACATCGAACACAGATctgaatactcagttcacaagttgttaacatatgcgtttactttcaaaaacctgacagcagaaggaaggaaaagaactcacCCAAGAATCCCTAAGAGTATAGGTAATGTTTCATCAGTACCATTATACAATGCCAATTCTTCAGCAGAAAATAACCTCTTTTGATActgaaacacaaacaaaccatcaaatcaaagacatgagacatcatcataacattggtttatctatttcaattctaattaaactcaagacccatacataacaaccacagagacaaacttcattggctatagaagaccctgtataattcaatcaaaaagtagcagaagctaaatagagattttgaattccagctaagctcgtgtctgtttggaattttcagaacagggaagttagttagaaagaagagcagttggtacTTGGTGTTGTGGAGACTTGAAGGAAGATCTGAAGTAGAGAGAGACGAGAACGACGATAAACGTGACGCCGACAAATGGAGATAAGAGAAACCTCCTCGCCGGAATCATcgattggatccgatttcacgtgtctgcgaaatttgctaattttttacaccactttttaggatgtgatgacttttttaactaaacaacaaaagttaaggtactaaactgtgatacttcaaatattcagagtgcgtttcataaattttttaagaccgaccttgaacacgataaattaataagataataatttttctgaaagtttgatgtgaatgtatggtctcatcaatatcataaattaataattgtataaatatattaactatatatatattatatgtaaaagaatttctttaaaatatattttaaaattttttttgcttaaattcatatttgttctttgtttgagtataattttcatattttattgtatcaaaatttttggtgttgttttctaaacattattattatgttttacttgtaactggttctaaatatatattttctacaatgaaagaaatcttataaaacaattttaaaagttaccaaattaggaaaatgtctgtataaattaataaatattaatttatcgataaattaatacctctctaaattaataaaatttcgcggtcccaacattattaatttatagaggttttattgtatatcgatttatcatttccctgaagattgattgcaagttcaaacatacaatggaaattacttatctgatacactttggttcagtcctaaataatggtctgatacggttcaattccagttaaaattccggttaagttccacgcaattcaagggattttgcttttcaaaaattccaaaattataaaatacacagctcacgagaataacaacaaatcgtggttttaacaacaacccaattacatcattacacgtgtaatatacctgccgttacatacagatgggataattttcgcttataaaatacagcatcaaacaccttcctcataacctagaaatcgaaaatatcagatctaaagaagagcgtcttcataaacgcccttcgtcttcttccctgcaaatctattttctagatctttctttcttccgcgaaacgaca'
primerfxn(mydna2)

('caggtcttctctgtagc', 'caggtcttctctgtagc', 58, 58)

('TGCATTTTTTGTAGCTTC', 'TGCATTTTTTGTAGCTTC', 56, 56)

('attcaaagatgtgtaatgg', 'attcaaagatgtgtaatgg', 55, 55)

In [16]:
def get_anneal(temp1, temp2):
    return min(temp1, temp2)+1

In [17]:
# Next step is to make a fxn that takes a list of dna and does everything to it (probably "for loop")
def PrimerMaker(mydnalist):
results = []
for dnas in mydnalist: 
results.append(primerfxn(dnas))
df = pd.DataFrame(results, columns = ['fwd_primer' , 'rev_primer', 'Fwd Primer Tm', 'Rev Primer Tm'])
df["anneal"] = df.apply(lambda x: get_anneal(x["Fwd Primer Tm"], x["Rev Primer Tm"]), axis=1)
return df

In [18]:
def bsai_regex(seq):
    my_matches = []
    match = re.finditer(r"(GGTCTC)|(GAGACC)", seq, flags=re.IGNORECASE) 
    for m in match:
        my_matches.append(m.span())
    return my_matches

def bsmbi_regex(seq):
    my_matches = []
    match = re.finditer(r"(CGTCTC)|(GAGACG)", seq, flags=re.IGNORECASE) 
    for m in match:
        my_matches.append(m.span())
    return my_matches 

def bbsi_regex(seq):
    my_matches = []
    match = re.finditer(r"(GAAGAC)|(GTCTTC)", seq, flags=re.IGNORECASE) 
    for m in match:
        my_matches.append(m.span())
    return my_matches

In [19]:
def recommender(l_bsai, l_bsmbi, l_bbsi):
    if len(l_bsai) == 0:
        return "BsaI"
    if len(l_bbsi) == 0:
        return "BbsI"
    if len(l_bsmbi) == 0:
        return "BsmbI"
    else:
        return "not good"

In [20]:
def type2s_wrapper(seq):
    bsai_list = bsai_regex(seq)
    bsmbi_list = bsmbi_regex(seq)
    bbsi_list = bbsi_regex(seq)
    return recommender(bsai_list, bsmbi_list, bbsi_list)

In [12]:
gene_df = pd.read_csv("updated_constitutive_promoters_from_izaiah.csv")

In [13]:
gene_df.head()

,TAIR_ID,DEseq2_means,DEseq2_SDs,gene_name,leaf,flower,fruit,root,avg_tpm,Gene Coordinates,5' UTR Length (bp),TAIR Link,Promoter 2kb upstream from ATG,Promoter B4 5' UTR,Notes
0,AT5G02500,15.914132,0.648128,MED37E,961,1308,639,2095,1250.75,Chr5: 553745 - 556623,289,https://www.arabidopsis.org/servlets/TairObjec...,caggtcttctctgtagctctgttacttctatcacagttatcgggta...,CAGAGCCTAAATTCGAATGAgaatgaggtttcatctttgttagttt...,
1,AT1G04410,14.202362,0.509645,MDH1,809,645,778,1651,970.75,Chr1: 1188995 - 1191458,191,https://www.arabidopsis.org/servlets/TairObjec...,TGCATTTTTTGTAGCTTCTTGGTCTGTAGCCAAGTATTTCCCAGAC...,tgaaatcagaacaaagataaaattgacctcaaaacacttttacgaa...,
2,AT4G14960,14.081451,0.617216,TUBA6,479,747,780,758,691.00,Chr4: 8548417 - 8550495,176,https://www.arabidopsis.org/servlets/TairObjec...,attcaaagatgtgtaatgggaggaacacagaaaatttcctcataac...,GCTAGATAACCCTTGTAACGAATCTGTAAGTCCATCTCctgtacaa...,
3,AT1G07890,13.633046,0.494965,APX1,276,707,729,730,610.50,Chr1: 2437725 - 2439665,12 (39),https://www.arabidopsis.org/servlets/TairObjec...,tagaaaaagctatctcattgtggatcaataggtccctaggaaccaa...,agaagaactcggatcaagacaggttttaagattaaaaaaggaagca...,"Has 6 different splice variants, this ascensio..."
4,AT5G09810,13.955694,0.402030,ACT7,364,698,638,608,577.00,Chr5: 3052062 - 3054691,6 (206),https://www.arabidopsis.org/servlets/TairObjec...,ttttgattatttaagttgccaaccaagctagaccactttccaactt...,atttttaaccataggcaaagaaatgactcaaatgagcaaacgagaa...,5' UTR split by Intron.


In [14]:
gene_df["2kb_recommend"] = gene_df["Promoter 2kb upstream from ATG "].apply(type2s_wrapper)
gene_df["utr_recommend"] = gene_df["Promoter B4 5\' UTR"].apply(type2s_wrapper)

In [16]:
twokb = gene_df["Promoter 2kb upstream from ATG "].to_list()
utr2kb = gene_df["Promoter B4 5\' UTR"].to_list()

In [17]:
primer2kb = PrimerMaker(twokb)
primerutr2kb = PrimerMaker(utr2kb)

In [19]:
df_2kb = pd.concat([gene_df.TAIR_ID, gene_df.gene_name, primer2kb.fwd_primer, primer2kb.rev_primer, primer2kb.anneal], axis=1)
df_utr2kb = pd.concat([gene_df.TAIR_ID, gene_df.gene_name, primerutr2kb.fwd_primer, primerutr2kb.rev_primer, primerutr2kb.anneal], axis=1)

In [23]:
# DF label output for primer maker is incorrect
# some primers are too cold
df_utr2kb.head()

,TAIR_ID,gene_name,fwd_primer,rev_primer,anneal
0,AT5G02500,MED37E,CAGAGCCTAAATTCGAA,CAGAGCCTAAATTCGAA,57
1,AT1G04410,MDH1,tgaaatcagaacaaagataaa,tgaaatcagaacaaagataaa,56
2,AT4G14960,TUBA6,GCTAGATAACCCTTGTAAC,GCTAGATAACCCTTGTAAC,58
3,AT1G07890,APX1,agaagaactcggatca,agaagaactcggatca,57
4,AT5G09810,ACT7,atttttaaccataggcaaag,atttttaaccataggcaaag,57


In [21]:
def label_maker(gene_name, anneal, fwd=True, custom_str=""):
    if fwd == True:
        return gene_name + "_F_" + str(anneal) + custom_str
    if fwd == False:
        return gene_name + "_R_" + str(anneal) + custom_str

In [25]:
df_2kb["fwd_label"] = df_2kb.apply(lambda x: label_maker(x["TAIR_ID"], x["anneal"]), axis=1)
df_2kb["rev_label"] = df_2kb.apply(lambda x: label_maker(x["TAIR_ID"], x["anneal"], fwd=False), axis=1)

df_utr2kb["fwd_label"] = df_utr2kb.apply(lambda x: label_maker(x["TAIR_ID"], x["anneal"], custom_str="utr"), axis=1)
df_utr2kb["rev_label"] = df_utr2kb.apply(lambda x: label_maker(x["TAIR_ID"], x["anneal"], custom_str="utr", fwd=False), axis=1)

In [22]:
def series_joiner(ser1, ser2, ser3):
    return pd.concat([ser1, ser2, ser3], axis=1)

In [28]:
df_2kb.head()
df_utr2kb.head()

,TAIR_ID,gene_name,fwd_primer,rev_primer,anneal,fwd_label,rev_label
0,AT5G02500,MED37E,caggtcttctctgtagc,caggtcttctctgtagc,59,AT5G02500_F_59,AT5G02500_R_59
1,AT1G04410,MDH1,TGCATTTTTTGTAGCTTC,TGCATTTTTTGTAGCTTC,57,AT1G04410_F_57,AT1G04410_R_57
2,AT4G14960,TUBA6,attcaaagatgtgtaatgg,attcaaagatgtgtaatgg,56,AT4G14960_F_56,AT4G14960_R_56
3,AT1G07890,APX1,tagaaaaagctatctcattgt,tagaaaaagctatctcattgt,58,AT1G07890_F_58,AT1G07890_R_58
4,AT5G09810,ACT7,ttttgattatttaagttgcca,ttttgattatttaagttgcca,57,AT5G09810_F_57,AT5G09810_R_57


,TAIR_ID,gene_name,fwd_primer,rev_primer,anneal,fwd_label,rev_label
0,AT5G02500,MED37E,CAGAGCCTAAATTCGAA,CAGAGCCTAAATTCGAA,57,AT5G02500_F_57utr,AT5G02500_R_57utr
1,AT1G04410,MDH1,tgaaatcagaacaaagataaa,tgaaatcagaacaaagataaa,56,AT1G04410_F_56utr,AT1G04410_R_56utr
2,AT4G14960,TUBA6,GCTAGATAACCCTTGTAAC,GCTAGATAACCCTTGTAAC,58,AT4G14960_F_58utr,AT4G14960_R_58utr
3,AT1G07890,APX1,agaagaactcggatca,agaagaactcggatca,57,AT1G07890_F_57utr,AT1G07890_R_57utr
4,AT5G09810,ACT7,atttttaaccataggcaaag,atttttaaccataggcaaag,57,AT5G09810_F_57utr,AT5G09810_R_57utr


In [29]:
fwd_idt_2kb = series_joiner(df_2kb.fwd_primer, df_2kb.fwd_label, gene_df["2kb_recommend"])
fwd_idt_utr2kb = series_joiner(df_utr2kb.fwd_primer, df_utr2kb.fwd_label, gene_df["utr_recommend"])

rev_idt_2kb = series_joiner(df_2kb.rev_primer, df_2kb.rev_label, gene_df["2kb_recommend"])
rev_idt_utr2kb = series_joiner(df_utr2kb.rev_primer, df_utr2kb.rev_label, gene_df["utr_recommend"])

In [23]:
BsaIer = lambda x, y: "GGTCTCg" + y + x 
BsmbIer = lambda x, y: "CGTCTCg" + y + x 
BbsIer = lambda x, y: "GAAGACtg" + y + x 

In [24]:
def type2s_appender(seq, recommend, overhang):
    if recommend == "BsaI":
        return BsaIer(seq, overhang)
    if recommend == "BsmbI":
        return BsmbIer(seq, overhang)
    if recommend == "BbsI":
        return BbsIer(seq, overhang)

In [34]:
fwd_idt_2kb.fwd_primer = fwd_idt_2kb.apply(lambda x: type2s_appender(x["fwd_primer"], x["2kb_recommend"], "gctt"), axis=1)
rev_idt_2kb.rev_primer = rev_idt_2kb.apply(lambda x: type2s_appender(x["rev_primer"], x["2kb_recommend"], "catt"), axis=1)

fwd_idt_utr2kb.fwd_primer = fwd_idt_utr2kb.apply(lambda x: type2s_appender(x["fwd_primer"], x["utr_recommend"], "gctt"), axis=1)
rev_idt_utr2kb.rev_primer = rev_idt_utr2kb.apply(lambda x: type2s_appender(x["rev_primer"], x["utr_recommend"], "catt"), axis=1)

In [36]:
fwd_idt_2kb.dropna(inplace=True)
fwd_idt_2kb.drop(columns="2kb_recommend", inplace=True)
fwd_idt_2kb.reset_index(inplace=True, drop=True)

rev_idt_2kb.dropna(inplace=True)
rev_idt_2kb.drop(columns="2kb_recommend", inplace=True)
rev_idt_2kb.reset_index(inplace=True, drop=True)

fwd_idt_utr2kb.dropna(inplace=True)
fwd_idt_utr2kb.drop(columns="utr_recommend", inplace=True)
fwd_idt_utr2kb.reset_index(inplace=True, drop=True)

rev_idt_utr2kb.dropna(inplace=True)
rev_idt_utr2kb.drop(columns="utr_recommend", inplace=True)
rev_idt_utr2kb.reset_index(inplace=True, drop=True)

In [25]:
def char_range(c1, c2):
    """Generates the characters from `c1` to `c2`, inclusive."""
    for c in range(ord(c1), ord(c2)+1):
        yield chr(c)

In [26]:
def generate_96_labs():
    for number in range(1,13):
        for letter in char_range("A", "H"):
            yield(letter+str(number))

In [27]:
def append_well_labs(fwd_frame, rev_frame):
    fwd_frame.rename(columns={"fwd_label": "Name", "fwd_primer": "Sequence"}, inplace=True)
    rev_frame.rename(columns={"rev_label": "Name", "rev_primer": "Sequence"}, inplace=True)
    my_lab = generate_96_labs()
    for idx, entry in fwd_frame.iterrows():
        curr_lab = fwd_frame.at[idx, "Well Position"] = next(my_lab)
    while curr_lab[0] != "H":
        curr_lab = next(my_lab)
    for idx, entry in rev_frame.iterrows():
        rev_frame.at[idx, "Well Position"] = next(my_lab)
    result = pd.concat([fwd_frame, rev_frame])
    return result[result.columns[::-1]]

In [52]:
twokb_96_order = append_well_labs(fwd_idt_2kb, rev_idt_2kb)

In [53]:
utrtwokb_96_order = append_well_labs(fwd_idt_utr2kb, rev_idt_utr2kb)

In [54]:
twokb_96_order

,Well Position,Name,Sequence
0,A1,AT5G02500_F_59,GGTCTCggcttcaggtcttctctgtagc
1,B1,AT1G04410_F_57,GAAGACtggcttTGCATTTTTTGTAGCTTC
2,C1,AT1G07890_F_58,GGTCTCggctttagaaaaagctatctcattgt
3,D1,AT5G09810_F_57,GAAGACtggcttttttgattatttaagttgcca
4,E1,AT3G48140_F_57,GGTCTCggcttatatcacaattccattattcg
5,F1,AT1G15690_F_57,GGTCTCggcttctttttcgttctaattgagt
6,G1,AT3G55440_F_57,GGTCTCggcttAGGCACGTAGGATATC
7,H1,AT4G39260_F_56,GGTCTCggcttTTGTCTGTTGCTCAAA
8,A2,AT4G40040_F_57,GGTCTCggcttTGAGAACAAATGCTCAA
9,B2,AT1G20693_F_56,GAAGACtggcttCTTGTAGCTTCCATCC


In [ ]:
twokb_96_order.to_excel("2kb_idt_order.xls", index=False, sheet_name="2kb_plate")
utrtwokb_96_order.to_excel("utr2kb_idt_order.xls", index=False, sheet_name="2kb_UTR_plate")

## To get comfortable manipulating DFs

1. .apply() function for DFs
2. iterrows() and iteritems() function
3. Just flat out making a copy of a column

In [ ]:
primer2kb["Name"] = "pIO"

In [ ]:
primer2kb.head()

In [ ]:
thing1 = ["pIO", "pAZ", "pIO",  "pIO",  "pIO"]

In [ ]:
for idx, item in enumerate(thing1):
    print(item + "_" +str(idx))

In [ ]:
for idx, item in primer2kb["Name"].iteritems():
    my_name = item + "_" +str(idx+1)
    primer2kb.at[idx, "Name"] = my_name


for idx, item in primer2kb.Name.iteritems():
    my_name = item + "_" +str(idx+1)
    primer2kb.at[idx, "Name"] = my_name

In [ ]:
fwd_frame = primer2kb[["Name", "fwd_primer"]]

rev_frame = primer2kb[["Name", "rev_primer"]]

In [ ]:
for idx, item in fwd_frame["Name"].iteritems():
    my_name = item + "F"
    fwd_frame.at[idx, "Name"] = my_name
    
for idx, item in rev_frame["Name"].iteritems():
    my_name = item + "R"
    rev_frame.at[idx, "Name"] = my_name

In [ ]:
fwd_frame.rename(columns={"fwd_primer": "Seq"}, inplace=True)
rev_frame.rename(columns={"rev_primer": "Seq"}, inplace=True)

In [ ]:
pd.concat([fwd_frame, rev_frame])

In [81]:
gene_df = test_df
gene_df.rename(columns={gene_df.columns[0]: "Name"}, inplace = True)
names = gene_df.iloc[:,0]
sequences = gene_df.iloc[:,1].to_list()

/home/stantasy/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [82]:
primer_df = PrimerMaker(sequences)

In [83]:
result = pd.concat([names, primer_df[["fwd_primer", "rev_primer", "anneal"]]], axis=1)

In [86]:
result["fwd_label"] = result.apply(lambda x: label_maker(x["Name"], x["anneal"]), axis=1)
result["rev_label"] = result.apply(lambda x: label_maker(x["Name"], x["anneal"], fwd=False), axis=1)

In [93]:
fwd_order = pd.DataFrame()
fwd_order["label"] = result["fwd_label"] 
fwd_order["seq"] = result["fwd_primer"]

rev_order = pd.DataFrame()
rev_order["label"] = result["rev_label"] 
rev_order["seq"] = result["rev_primer"]

In [94]:
idt_order = pd.concat([fwd_order,rev_order])

In [100]:
gene_df["recommend"] = gene_df.iloc[:,1].apply(type2s_wrapper)

/home/stantasy/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [102]:
gg_df = gene_df

In [103]:
gg_df

,Name,Promoter 2kb upstream from ATG,recommend
0,AT5G02500,caggtcttctctgtagctctgttacttctatcacagttatcgggta...,BsaI
1,AT1G04410,TGCATTTTTTGTAGCTTCTTGGTCTGTAGCCAAGTATTTCCCAGAC...,BbsI
2,AT4G14960,attcaaagatgtgtaatgggaggaacacagaaaatttcctcataac...,not good
3,AT1G07890,tagaaaaagctatctcattgtggatcaataggtccctaggaaccaa...,BsaI
4,AT5G09810,ttttgattatttaagttgccaaccaagctagaccactttccaactt...,BbsI
5,AT3G48140,atatcacaattccattattcgaataaggactatcaaattttatcta...,BsaI
6,AT1G15690,ctttttcgttctaattgagtaattcataaacgtttaactaattcga...,BsaI
7,AT3G55440,AGGCACGTAGGATATCGGGATTCACATCGAAGATCTTGACGCTATC...,BsaI
8,AT5G39570,CCAAACCCTGGATCTCTCTCCACCGTCTATCCTCGTAATCATACCA...,not good
9,AT4G39260,TTGTCTGTTGCTCAAACGTTGACTCTAACAACAAGAGGAGGAATAA...,BsaI


In [104]:
primer_df

,fwd_primer,rev_primer,Fwd Primer Tm,Rev Primer Tm,anneal
0,caggtcttctctgtagc,caggtcttctctgtagc,58,58,59
1,TGCATTTTTTGTAGCTTC,TGCATTTTTTGTAGCTTC,56,56,57
2,attcaaagatgtgtaatgg,attcaaagatgtgtaatgg,55,55,56
3,tagaaaaagctatctcattgt,tagaaaaagctatctcattgt,57,57,58
4,ttttgattatttaagttgcca,ttttgattatttaagttgcca,56,56,57
5,atatcacaattccattattcg,atatcacaattccattattcg,56,56,57
6,ctttttcgttctaattgagt,ctttttcgttctaattgagt,56,56,57
7,AGGCACGTAGGATATC,AGGCACGTAGGATATC,56,56,57
8,CCAAACCCTGGATCT,CCAAACCCTGGATCT,56,56,57
9,TTGTCTGTTGCTCAAA,TTGTCTGTTGCTCAAA,55,55,56


In [108]:
result = pd.concat([names, gene_df["recommend"], primer_df[["fwd_primer", "rev_primer", "anneal"]]], axis=1)
result["fwd_label"] = result.apply(lambda x: label_maker(x["Name"], x["anneal"]), axis=1)
result["rev_label"] = result.apply(lambda x: label_maker(x["Name"], x["anneal"], fwd=False), axis=1)

In [118]:
fwd_order = pd.DataFrame()
fwd_order["label"] = result["fwd_label"] 
fwd_order["seq"] = result["fwd_primer"]
fwd_order["recommend"] = result["recommend"]

rev_order = pd.DataFrame()
rev_order["label"] = result["rev_label"] 
rev_order["seq"] = result["rev_primer"]
rev_order["recommend"] = result["recommend"]

In [119]:
fwd_order.seq = fwd_order.apply(lambda x: type2s_appender(x["seq"], x["recommend"], "gctt"), axis=1)
rev_order.seq = rev_order.apply(lambda x: type2s_appender(x["seq"], x["recommend"], "catt"), axis=1)

In [121]:
fwd_order.dropna(inplace=True)
fwd_order.drop(columns="recommend", inplace=True)
fwd_order.reset_index(inplace=True, drop=True)

rev_order.dropna(inplace=True)
rev_order.drop(columns="recommend", inplace=True)
rev_order.reset_index(inplace=True, drop=True)

In [123]:
rev_order

,label,seq
0,AT5G02500_R_59,GGTCTCgcattcaggtcttctctgtagc
1,AT1G04410_R_57,GAAGACtgcattTGCATTTTTTGTAGCTTC
2,AT1G07890_R_58,GGTCTCgcatttagaaaaagctatctcattgt
3,AT5G09810_R_57,GAAGACtgcattttttgattatttaagttgcca
4,AT3G48140_R_57,GGTCTCgcattatatcacaattccattattcg
5,AT1G15690_R_57,GGTCTCgcattctttttcgttctaattgagt
6,AT3G55440_R_57,GGTCTCgcattAGGCACGTAGGATATC
7,AT4G39260_R_56,GGTCTCgcattTTGTCTGTTGCTCAAA
8,AT4G40040_R_57,GGTCTCgcattTGAGAACAAATGCTCAA
9,AT1G20693_R_56,GAAGACtgcattCTTGTAGCTTCCATCC


In [135]:
a = 2
b = 2

if a is None or b is None:
    print("yes")

In [ ]:
m = re.search(r"[^ATGC]", dna)

# m is now a match object
if m:
    print("ambiguous base found!")
    ambig = m.group()
    print("the base is " + ambig)

In [145]:
def bad_base(seq):
    my_matches = []
    match = re.finditer(r"[^ATGC]", seq, flags=re.IGNORECASE) 
    for m in match:
        my_matches.append(m.span())
    return my_matches

In [147]:
bad_base("GAAGAGAGgA")

[]

In [153]:
idt_order.reset_index(inplace=True, drop=True)

In [ ]:
idt_order.loc[:,]

In [160]:
base_check = idt_order.seq.apply(bad_base)
base_check_count = idt_order.seq.apply(bad_base).apply(len).to_list()

In [167]:
bad_base_list = []
for idx, count in enumerate(base_check_count):
    if count != 0:
        bad_base_list.append(idx+1)
if len(bad_base_list) == 0:
    print(idt_order)
else:
    print("Error! Ambiguous bases found at seq numbers: ", bad_base_list)

             label                               seq
0   AT5G02500_F_59                 caggtcttctctgtagc
1   AT1G04410_F_57                TGCATTTTTTGTAGCTTC
2   AT4G14960_F_56               attcaaagatgtgtaatgg
3   AT1G07890_F_58             tagaaaaagctatctcattgt
4   AT5G09810_F_57             ttttgattatttaagttgcca
5   AT3G48140_F_57             atatcacaattccattattcg
6   AT1G15690_F_57              ctttttcgttctaattgagt
7   AT3G55440_F_57                  AGGCACGTAGGATATC
8   AT5G39570_F_57                   CCAAACCCTGGATCT
9   AT4G39260_F_56                  TTGTCTGTTGCTCAAA
10  AT4G40040_F_57                 TGAGAACAAATGCTCAA
11  AT1G15270_F_58                ATGTACCACATTACTGGA
12  AT1G20693_F_56                  CTTGTAGCTTCCATCC
13  AT5G11740_F_57              GACATTACAGAAAACTTCTT
14  AT4G40030_F_57             attagaaaacttgagaaacac
15  AT2G33040_F_56                cctgtgaaactattgttc
16  AT3G15730_F_56             cttttgggttttctattttag
17  AT3G13920_F_57                  tcagagatcg

In [169]:
def append_well_labs(fwd_frame, rev_frame):
    fwd_frame.rename(columns={"label": "Name", "seq": "Sequence"}, inplace=True)
    rev_frame.rename(columns={"label": "Name", "seq": "Sequence"}, inplace=True)
    my_lab = generate_96_labs()
    for idx, entry in fwd_frame.iterrows():
        curr_lab = fwd_frame.at[idx, "Well Position"] = next(my_lab)
    while curr_lab[0] != "H":
        curr_lab = next(my_lab)
    for idx, entry in rev_frame.iterrows():
        rev_frame.at[idx, "Well Position"] = next(my_lab)
    result = pd.concat([fwd_frame, rev_frame])
    return result[result.columns[::-1]]

In [170]:
append_well_labs(fwd_order, rev_order)

,Well Position,Sequence,Name
0,A1,GGTCTCggcttcaggtcttctctgtagc,AT5G02500_F_59
1,B1,GAAGACtggcttTGCATTTTTTGTAGCTTC,AT1G04410_F_57
2,C1,GGTCTCggctttagaaaaagctatctcattgt,AT1G07890_F_58
3,D1,GAAGACtggcttttttgattatttaagttgcca,AT5G09810_F_57
4,E1,GGTCTCggcttatatcacaattccattattcg,AT3G48140_F_57
5,F1,GGTCTCggcttctttttcgttctaattgagt,AT1G15690_F_57
6,G1,GGTCTCggcttAGGCACGTAGGATATC,AT3G55440_F_57
7,H1,GGTCTCggcttTTGTCTGTTGCTCAAA,AT4G39260_F_56
8,A2,GGTCTCggcttTGAGAACAAATGCTCAA,AT4G40040_F_57
9,B2,GAAGACtggcttCTTGTAGCTTCCATCC,AT1G20693_F_56


In [174]:
for idx, item in idt_order.iterrows():
    print(item[0]+","+item[1])

AT5G02500_F_59,caggtcttctctgtagc
AT1G04410_F_57,TGCATTTTTTGTAGCTTC
AT4G14960_F_56,attcaaagatgtgtaatgg
AT1G07890_F_58,tagaaaaagctatctcattgt
AT5G09810_F_57,ttttgattatttaagttgcca
AT3G48140_F_57,atatcacaattccattattcg
AT1G15690_F_57,ctttttcgttctaattgagt
AT3G55440_F_57,AGGCACGTAGGATATC
AT5G39570_F_57,CCAAACCCTGGATCT
AT4G39260_F_56,TTGTCTGTTGCTCAAA
AT4G40040_F_57,TGAGAACAAATGCTCAA
AT1G15270_F_58,ATGTACCACATTACTGGA
AT1G20693_F_56,CTTGTAGCTTCCATCC
AT5G11740_F_57,GACATTACAGAAAACTTCTT
AT4G40030_F_57,attagaaaacttgagaaacac
AT2G33040_F_56,cctgtgaaactattgttc
AT3G15730_F_56,cttttgggttttctattttag
AT3G13920_F_57,tcagagatcgagatcg
AT1G78900_F_57,ggtttgcaatatttgaca
AT1G11910_F_58,CATGAGAAGGGAGAAGA
AT1G75950_F_56,gaagtgataaaaatataatattatataaaaat
AT4G13930_F_56,ttctaatatttaatatttttgtattcatat
AT5G59880_F_65,cgggttagcgggacg
AT5G02500_R_59,caggtcttctctgtagc
AT1G04410_R_57,TGCATTTTTTGTAGCTTC
AT4G14960_R_56,attcaaagatgtgtaatgg
AT1G07890_R_58,tagaaaaagctatctcattgt
AT5G09810_R_57,ttttgattatttaagttgcca
AT3G48140_R_5